# Segmenting and Clustering Neighborhoods in Toronto

Importing Required Libraries

In [35]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print("Libraries imported successfully.")

Libraries imported successfully.


Reading data from wiki page

In [36]:
data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(data_url)
response

<Response [200]>

Using BeautifulSoup Scraping List of Postal Codes of Given Wikipedia Page

In [37]:
soup_obj = BeautifulSoup(response.text, 'xml')
table_obj=soup_obj.find('table')

Parsing table data row and column wise and Ignore cells with a borough that is Not assigned

In [38]:
table_contents=[]
for row in table_obj.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

Creating Dataframe which consists of three columns(PostalCode, Borough, and Neighborhood)

In [39]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Print dataframe data

In [40]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Shape of dataframe

In [41]:
df.shape

(103, 3)

Reading geo data from CSV file into dataframe

In [42]:
lat_long_data_csv_file_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
lat_long_data_df = pd.read_csv(lat_long_data_csv_file_url)
lat_long_data_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging both geo data and postal data on postal code

In [43]:
lat_long_data_merged_df = pd.merge(df, lat_long_data_df, left_on='PostalCode', right_on='Postal Code')
lat_long_data_final_df = lat_long_data_merged_df[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
lat_long_data_final_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Filtering data on Borough that contain the word Toronto

In [44]:
borough_toronto_df = lat_long_data_final_df[lat_long_data_final_df['Borough'].str.contains('Toronto',regex=False)]
borough_toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Visualizing all the neighborhoods from the above data frame using folium

In [45]:
# Toronto map
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# adding marker to map
for lat,lng,borough,neighbourhood in zip(borough_toronto_df['Latitude'],borough_toronto_df['Longitude'],borough_toronto_df['Borough'],borough_toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

Using K-Means clustering algorithm for the clustering of the neighborhoods

In [46]:
k=5
toronto_cluster_data = borough_toronto_df.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_cluster_data)
kmeans.labels_
borough_toronto_df.insert(0, 'Cluster Labels', kmeans.labels_)
borough_toronto_df.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Visualizing all the 5 neighborhoods cluster from the above data frame using folium

In [47]:
# Toronto map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(borough_toronto_df['Latitude'], borough_toronto_df['Longitude'], borough_toronto_df['Neighborhood'], borough_toronto_df['Cluster Labels']):
    label = folium.Popup(' Cluster - ' + str(cluster) + ', '+ neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters